# Obter dados no drive

In [ ]:
!pip install tensorflow_model_optimization

In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)
from google.colab import drive
import pandas as pd
import random
import os
import time
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, Callback
from glob import glob
from sklearn.model_selection import train_test_split
import numpy as np
from keras import layers, regularizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow_model_optimization.sparsity import keras as sparsity
from tensorflow.keras.applications import MobileNetV2

In [ ]:
drive.mount('/content/drive')

# Carregamento de dados - Sem Data Augmentation


In [ ]:
images = []
masks = []

for path in glob('/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli/cropped_images/*/*'):
  images.append(path + '/image.tif')
  masks.append(path + '/mask.png')

In [ ]:
# Função para carregar e pré-processar uma imagem e sua máscara
def load_and_preprocess_image(image_path, mask_path, target_size):

    image = load_img(image_path, target_size=target_size)
    image = img_to_array(image) / 255.0  # Normalização entre 0 e 1

    mask = load_img(mask_path, target_size=target_size, color_mode='grayscale')
    mask = img_to_array(mask) / 255.0  # Normalização entre 0 e 1

    return image, mask

In [ ]:
with tf.device('/gpu:0'):
    # Lista para armazenar imagens e máscaras pré-processadas
    images_processed = []
    masks_processed = []

    # Carregar e pré-processar todas as imagens e máscaras
    for img_path, mask_path in zip(images, masks):
        img, mask = load_and_preprocess_image(img_path, mask_path, target_size=(256, 256))
        images_processed.append(img)
        masks_processed.append(mask)

    # Converter para arrays numpy
    images_processed = np.array(images_processed)
    masks_processed = np.array(masks_processed)


In [ ]:
images_processed

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(images_processed, masks_processed, test_size=0.3, random_state=42)

In [ ]:
images_processed.shape, masks_processed.shape

# Definição de parâmetros

In [ ]:
class CyclicLR(Callback):
    def __init__(self, base_lr=1e-4, max_lr=1e-3, step_size=2000., mode='triangular'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.iterations = 0
        self.history = {}

    def clr(self):
        cycle = np.floor(1 + self.iterations / (2 * self.step_size))
        x = np.abs(self.iterations / self.step_size - 2 * cycle + 1)
        lr = self.base_lr + (self.max_lr - self.base_lr) * max(0, (1 - x))
        if self.mode == 'triangular2':
            lr = lr / float(2 ** (cycle - 1))
        elif self.mode == 'exp_range':
            lr = lr * (0.999 ** self.iterations)
        return lr

    def on_train_begin(self, logs=None):
        logs = logs or {}
        tf.keras.backend.set_value(self.model.optimizer.lr, self.base_lr)

    def on_batch_end(self, batch, logs=None):
        self.iterations += 1
        lr = self.clr()
        tf.keras.backend.set_value(self.model.optimizer.lr, lr)
        self.history.setdefault('lr', []).append(lr)
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

# Função de callbacks
def get_callbacks():
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
    clr = CyclicLR(base_lr=1e-4, max_lr=1e-3, step_size=2000., mode='triangular2')
    pruning = sparsity.UpdatePruningStep()
    return [pruning, early_stopping, reduce_lr, clr]


In [ ]:
# Função para calcular a sigmoide e converter para 0 ou 1 o output
class ThresholdLayer(tf.keras.layers.Layer):
    def call(self, inputs):
        return tf.where(inputs < 0.5, 0.0, 1.0)

# Função para calcular o Dice Coefficient
def dice_coefficient(y_train, y_val):
    smooth = 1e-6
    intersection = tf.reduce_sum(y_train * y_val)
    dice_coefficient = (2. * intersection + smooth) / (tf.reduce_sum(y_train) + tf.reduce_sum(y_val) + smooth)
    return dice_coefficient

# Função de perda de Dice
def dice_loss(y_train, y_val):
    return 1 - dice_coefficient(y_train, y_val)

# Função para calcular a penalidade adicional
def penalty_loss(y_train, y_val, penalty_weight):
    # Calcular a penalidade considerando a diferença entre y_train e y_val
    penalty = tf.reduce_sum(tf.abs(y_train - y_val))
    # Multiplicar a penalidade pelo peso da penalidade
    weighted_penalty = penalty_weight * penalty
    return weighted_penalty

# Função de perda combinada
def combined_loss(y_train, y_val, penalty_weight):
    # Perda padrão (por exemplo, perda de entropia cruzada binária)
    standard_loss = tf.keras.losses.binary_crossentropy(y_train, y_val)
    # Dice Loss
    dice = dice_loss(y_train, y_val)
    # Penalidade adicional
    penalty = penalty_loss(y_train, y_val, penalty_weight)
    # Perda total = perda padrão + penalidade + Dice Loss
    total_loss = standard_loss + penalty + dice
    return total_loss

# Métrica de acurácia customizada
def custom_accuracy(y_train, y_val):
    # Calcular a acurácia considerando uma tolerância de 0.5 na predição
    y_val_binary = tf.round(y_val)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(y_train, y_val_binary), tf.float32))
    return accuracy

# Modelo Pré-treinado MobileNetV2

O modelo que utilizamos para fazer o transfer learning é o MobileNetV2, que pode ser encontrado o artigo de sua publicação no link:
https://doi.org/10.48550/arXiv.1801.04381

In [ ]:
# Carregar o modelo MobileNetV2 pré-treinado
pre_trained_model = MobileNetV2(weights='imagenet', include_top=False)

# Visualizar a estrutura do modelo
pre_trained_model.summary()

# Modelo U-Net

A U-Net, apresentada no artigo "U-Net: Convolutional Networks for Biomedical Image Segmentation" [1], é uma arquitetura de rede neural convolucional (CNN) projetada especificamente para tarefas de segmentação de imagens biomédicas, sua característica principal é sua arquitetura em formato de "U", que inclui conexões de salto entre as camadas correspondentes da contração e expansão, permite que a rede mantenha uma alta precisão, mesmo em detalhes minuciosos.

O modelo alcançou resultados impressionantes em várias métricas de avaliação em diferentes desafios de segmentação de imagens biomédicas como o ISBI, mas também foi adaptada para uma ampla gama de aplicações, incluindo segmentação de imagens satelitais, como em nosso caso.

Referências:

[1] Ronneberger, O., Fischer, P., & Brox, T. (2015). U-Net: Convolutional Networks for Biomedical Image Segmentation. In Proceedings of the International Conference on Medical Image Computing and Computer-Assisted Intervention (MICCAI). https://arxiv.org/abs/1505.04597

In [ ]:
class UNet:
    def __init__(self, input_shape, num_filters, kernel_size, dropout_rate, val_reg):
        self.input_shape = input_shape
        self.num_filters = num_filters
        self.kernel_size = kernel_size
        self.dropout_rate = dropout_rate
        self.val_reg = val_reg

    def build_model(self):
        inputs = tf.keras.Input(shape=self.input_shape)
        reg = regularizers.L2(self.val_reg)

        # Pruning parameters
        pruning_params = {'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.0,
                                                                       final_sparsity=0.5,
                                                                       begin_step=0,
                                                                       end_step=1000,
                                                                       frequency=100)}

        # Encoder (contraction path)
        base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=inputs)
        base_model.trainable = False

        # Extract specific layers for connections
        conv2 = base_model.get_layer('block_1_expand_relu').output
        conv3 = base_model.get_layer('block_3_expand_relu').output
        decoded = base_model.get_layer('block_6_expand_relu').output

        # Decoder (expansion path)
        up5 = sparsity.prune_low_magnitude(layers.Conv2DTranspose(self.num_filters[2], (2, 2), strides=(2, 2), padding='same'), **pruning_params)(decoded)
        up5 = layers.BatchNormalization()(up5)
        up5 = layers.Activation('relu')(up5)
        merge5 = layers.concatenate([conv3, up5], axis=3)
        conv5 = layers.Conv2D(self.num_filters[2], self.kernel_size, activation='relu', padding='same', kernel_regularizer=reg)(merge5)
        conv5 = layers.Conv2D(self.num_filters[2], self.kernel_size, activation='relu', padding='same', kernel_regularizer=reg)(conv5)

        up6 = sparsity.prune_low_magnitude(layers.Conv2DTranspose(self.num_filters[1], (2, 2), strides=(2, 2), padding='same'), **pruning_params)(conv5)
        up6 = layers.BatchNormalization()(up6)
        up6 = layers.Activation('relu')(up6)
        merge6 = layers.concatenate([conv2, up6], axis=3)
        conv6 = layers.Conv2D(self.num_filters[1], self.kernel_size, activation='relu', padding='same', kernel_regularizer=reg)(merge6)
        conv6 = layers.Conv2D(self.num_filters[1], self.kernel_size, activation='relu', padding='same', kernel_regularizer=reg)(conv6)

        up7 = sparsity.prune_low_magnitude(layers.Conv2DTranspose(self.num_filters[0], (2, 2), strides=(2, 2), padding='same'), **pruning_params)(conv6)
        up7 = layers.BatchNormalization()(up7)
        up7 = layers.Activation('relu')(up7)
        conv7 = layers.Conv2D(self.num_filters[0], self.kernel_size, activation='relu', padding='same', kernel_regularizer=reg)(up7)
        conv7 = layers.Conv2D(self.num_filters[0], self.kernel_size, activation='relu', padding='same', kernel_regularizer=reg)(conv7)

        outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(conv7)

        model = tf.keras.Model(inputs=inputs, outputs=outputs)
        return model

    def compile_and_train(self, X_train, y_train, X_val, y_val, max_epochs, batch_size):
        model = self.build_model()
        callbacks = get_callbacks()

        # Compilar o modelo
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                      loss=lambda y_train, y_val: combined_loss(y_train, y_val, 0.001),
                      metrics=[custom_accuracy])

        model.summary()

        start_time = time.time()

        # Treinar o modelo
        H = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                      epochs=max_epochs, batch_size=batch_size, callbacks=callbacks)

        # Salvar o tempo de treino
        training_time_gpu = time.time() - start_time

        return H, training_time_gpu, model


In [ ]:
def evaluate_model(model, X_val, y_val, H, training_time_gpu, max_epochs):
    plt.style.use("ggplot")
    plt.figure(figsize=(12, 10))

    start_time = time.time()

    results = model.evaluate(X_val, y_val)

    inference_time_gpu = time.time() - start_time

    print("Test Loss:", results[0])
    print("Test Accuracy:", results[1])

    # Prever máscaras usando o modelo
    predicted_masks = model.predict(X_val)

    # Obter métricas de precisão e perda do treinamento
    acc = H.history['custom_accuracy']
    val_acc = H.history['val_custom_accuracy']
    loss = H.history['loss']
    val_loss = H.history['val_loss']

    epochs = range(1, len(acc) + 1)

    # Plotar precisão do conjunto
    plt.subplot(2, 2, 3)
    plt.plot(epochs, acc, 'r', label='Precisão do Conjunto de Treino')
    plt.plot(epochs, val_acc, 'b', label='Precisão do Conjunto de Validação')
    plt.title('Precisão do Conjunto de Treino e Validação')
    plt.xlabel('Épocas')
    plt.ylabel('Precisão')
    plt.legend()

    # Plotar perda do conjunto
    plt.subplot(2, 2, 4)
    plt.plot(epochs, loss, 'r', label='Perda do Conjunto de Treino')
    plt.plot(epochs, val_loss, 'b', label='Perda do Conjunto de Validação')
    plt.title('Perda do Conjunto de Treino e Validação')
    plt.xlabel('Épocas')
    plt.ylabel('Perda')
    plt.legend()

    plt.tight_layout()
    plt.show()

    print('Tempo de treino (segundos): ', training_time_gpu)
    print('Tempo de treino por época (segundos): ', training_time_gpu / max_epochs)
    print('Tempo de inferência (segundos): ', inference_time_gpu)

    # Plotando o tempo de treinamento e inferência
    plt.figure(figsize=(6, 4))
    plt.bar(['Training'], [training_time_gpu], color='orange', label='Training Time')
    plt.bar(['Inference'], [inference_time_gpu], color='blue', label='Inference Time')
    plt.ylabel('Time (seconds)')
    plt.title('Training and Inference Time')
    plt.legend()
    plt.show()

    return predicted_masks, inference_time_gpu

# Transfer Learning - MobileNetV2

## Treino e inferência com CPU

Para esse teste com a CPU, é preciso apenas alocar uma máquina qualquer no colab, executar as células do início do notebook até o código abaixo.

In [ ]:
max_epochs = 10

unet = UNet(input_shape = (256, 256, 3),
            num_filters=(16, 32, 64, 128),
            kernel_size = 3,
            dropout_rate=0.1,
            val_reg =0.01)


# Treinando o modelo
H1Net, training_time_gpu1Net, model1Net = unet.compile_and_train(X_train, y_train, X_val, y_val,
                                                     max_epochs = max_epochs,
                                                     batch_size = 16)


# Avaliando o modelo
predicted_masks1Net, inference_time_gpu1Net = evaluate_model(model1Net, X_val, y_val, H1Net, training_time_gpu1Net, max_epochs=max_epochs)


## Treino e inferência com GPU
Para fazer esse teste conecte uma máquina do colab que possua GPU. Embaixo de compartilhar, no canto superior direito está escrito "Conect". Clique na seta para baixo ao lado de conectar. Selecione "Change runtime type" e selecione uma máquina que tenha GPU no nome. A que utilizamos foi a A100 GPU. Após isso, execute as células do início do notebook até o código abaixo.

In [ ]:
max_epochs = 10

unet = UNet(input_shape = (256, 256, 3),
            num_filters=(16, 32, 64, 128),
            kernel_size = 3,
            dropout_rate=0.1,
            val_reg =0.01)

with tf.device('/gpu:0'):
  # Treinando o modelo
  H1Net, training_time_gpu1Net, model1Net = unet.compile_and_train(X_train, y_train, X_val, y_val,
                                                      max_epochs = max_epochs,
                                                      batch_size = 16)


  # Avaliando o modelo
  predicted_masks1Net, inference_time_gpu1Net = evaluate_model(model1Net, X_val, y_val, H1Net, training_time_gpu1Net, max_epochs=max_epochs)


## Amostragem das máscaras e IoU

In [ ]:
with tf.device('/gpu:0'):
    # Gerar as saídas do modelo para um conjunto de entradas de teste
    saidas_modelo1Net = model1Net.predict(X_val)

    # Iterar sobre cada saída do modelo
    for i in range(len(X_val)):
        # Obter a entrada correspondente e a saída real
        img_entrada = X_val[i]
        img_saida_real = y_val[i]

        # Obter a saída gerada pelo modelo
        #img_saida_modelo1Net = saidas_modelo1Net[i]
        img_saida_modelo1Net = np.where(saidas_modelo1Net[i] < 0.5, 0, 1)

        # Mostrar as imagens
        plt.figure(figsize=(12, 4))

        plt.subplot(1, 3, 1)
        plt.imshow(img_entrada.squeeze(), cmap='gray')
        plt.title('Entrada')
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.imshow(img_saida_real.squeeze(), cmap='gray')
        plt.title('Saída Esperada')
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.imshow(img_saida_modelo1Net.squeeze(), cmap='gray')
        plt.title('Saída do Modelo - GPU')
        plt.axis('off')

        plt.show()

In [ ]:
with tf.device('/gpu:0'):
    # Métricas do parceiro de Projeto:

    # Lista para armazenar os scores de IoU
    iou_scores = []
    # Calcular IoUs e determinar predições corretas
    correct_predictions = 0
    iou_threshold = 0.5
    for mask, result in zip(y_val, saidas_modelo1Net):
        intersection = np.logical_and(mask, result)
        union = np.logical_or(mask, result)
        iou_score = np.sum(intersection) / np.sum(union) if np.sum(union) != 0 else 0
        iou_scores.append(iou_score)
        # Verificar se a predição é considerada correta (IoU >= threshold)
        if iou_score >= iou_threshold:
            correct_predictions += 1
        print('IoU é: ' + str(iou_score))
    # Calcular a média dos IoUs
    iou_mean = np.mean(iou_scores)
    print('Média dos IoU - GPU:', iou_mean)
    # Calcular Coverage Ratio (CovR)
    total_predictions = len(iou_scores)
    covr = correct_predictions / total_predictions if total_predictions > 0 else 0
    print('Coverage Ratio (CovR) - GPU:', covr)

# Testes com novas imagens

## Importando as imagens

In [ ]:
import os
import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

def load_masks(masks_dir, target_size=(600, 600), crop_size=(200, 200), final_size=(256, 256)):
    masks = []
    mask_filenames = []
    ordered_masks = sorted(os.listdir(masks_dir))
    for mask_name in ordered_masks:
        mask_path = os.path.join(masks_dir, mask_name)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is not None:
            if mask.shape[:2] != target_size:
                mask = cv2.resize(mask, target_size[::-1], interpolation=cv2.INTER_NEAREST)
            cropped_masks = crop_image(mask, crop_size)
            resized_masks = [cv2.resize(crop, final_size[::-1], interpolation=cv2.INTER_NEAREST) for crop in cropped_masks]
            normalized_masks = [crop / 255.0 for crop in resized_masks]
            masks.extend(normalized_masks)
            mask_filenames.extend([f"{os.path.splitext(mask_name)[0]}_crop_{i}" for i in range(len(normalized_masks))])
        else:
            print(f"Failed to load mask: {mask_path}")
    return masks, mask_filenames

def load_images(image_dir, target_size=(600, 600), crop_size=(200, 200), final_size=(256, 256)):
    images = []
    image_filenames = []
    ordered_images = sorted(os.listdir(image_dir))
    for image_name in ordered_images:
        image_path = os.path.join(image_dir, image_name)
        image = cv2.imread(image_path)
        if image is not None:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            if image.shape[:2] != target_size:
                image = cv2.resize(image, target_size[::-1])
            cropped_images = crop_image(image, crop_size)
            resized_images = [cv2.resize(crop, final_size[::-1]) for crop in cropped_images]
            normalized_images = [crop / 255.0 for crop in resized_images]
            images.extend(normalized_images)
            image_filenames.extend([f"{os.path.splitext(image_name)[0]}_crop_{i}" for i in range(len(normalized_images))])
        else:
            print(f"Failed to load image: {image_path}")
    return images, image_filenames

def crop_image(image, crop_size=(200, 200)):
    crops = []
    for i in range(0, image.shape[0], crop_size[0]):
        for j in range(0, image.shape[1], crop_size[1]):
            crop = image[i:i+crop_size[0], j:j+crop_size[1]]
            if crop.shape[0] == crop_size[0] and crop.shape[1] == crop_size[1]:
                crops.append(crop)
    return crops

# Paths
masks_dir = '/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli_test/masks'
image_dir = '/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli_test/tci_tifs'

# Load images and masks
masks_test, mask_filenames = load_masks(masks_dir)
images_test, image_filenames = load_images(image_dir)


In [ ]:
# verificando o tamanho correto
len(images_test)

In [ ]:
# Ensure the images are in the correct format for model prediction
images_test = np.array(images_test)

# Gerar as saídas do modelo para um conjunto de entradas de teste
saidas_novas_imagens_modelo = model1Net.predict(images_test)

In [ ]:
with tf.device('/gpu:0'):
    # Iterar sobre cada saída do modelo
    for i in range(len(images_test)):
        # Obter a entrada correspondente e a saída real
        img_entrada = images_test[i]
        img_saida_real = masks_test[i]

        # Obter a saída gerada pelo modelo
        img_saida_modelo = saidas_novas_imagens_modelo[i]

        # Mostrar as imagens
        plt.figure(figsize=(12, 4))

        plt.subplot(1, 3, 1)
        plt.imshow(img_entrada.squeeze(), cmap='gray')
        plt.title('Entrada')
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.imshow(img_saida_real.squeeze(), cmap='gray')
        plt.title('Saída Esperada')
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.imshow(img_saida_modelo.squeeze(), cmap='gray')
        plt.title('Saída do Modelo - GPU')
        plt.axis('off')

        plt.show()

In [ ]:
with tf.device('/gpu:0'):
    # Métricas do parceiro de Projeto:

    # Lista para armazenar os scores de IoU
    iou_scores = []
    # Calcular IoUs e determinar predições corretas
    correct_predictions = 0
    iou_threshold = 0.5
    for mask, result in zip(masks_test, saidas_novas_imagens_modelo):
        intersection = np.logical_and(mask, result)
        union = np.logical_or(mask, result)
        iou_score = np.sum(intersection) / np.sum(union) if np.sum(union) != 0 else 0
        iou_scores.append(iou_score)
        # Verificar se a predição é considerada correta (IoU >= threshold)
        if iou_score >= iou_threshold:
            correct_predictions += 1
        print('IoU é: ' + str(iou_score))
    # Calcular a média dos IoUs
    iou_mean = np.mean(iou_scores)
    print('Média dos IoU - GPU:', iou_mean)
    # Calcular Coverage Ratio (CovR)
    total_predictions = len(iou_scores)
    covr = correct_predictions / total_predictions if total_predictions > 0 else 0
    print('Coverage Ratio (CovR) - GPU:', covr)

# Pós processamento

In [ ]:
!pip install git+https://github.com/lucasb-eyer/pydensecrf.git

import numpy as np
import cv2
import pydensecrf.densecrf as dcrf
from pydensecrf.utils import unary_from_softmax
from sklearn.metrics import jaccard_score
from tqdm import tqdm

def postprocess_mask_with_crf(image, mask, crf_params):
    softmax = np.stack([1 - mask, mask], axis=-1)
    softmax = np.ascontiguousarray(softmax.transpose(2, 0, 1))
    image = np.ascontiguousarray(image)
    image_uint8 = (image * 255).astype(np.uint8)
    d = dcrf.DenseCRF2D(image.shape[1], image.shape[0], 2)
    unary = unary_from_softmax(softmax)
    d.setUnaryEnergy(unary)
    d.addPairwiseGaussian(sxy=crf_params['sxy'], compat=crf_params['compat'])
    d.addPairwiseBilateral(sxy=crf_params['sxy_bilateral'], srgb=crf_params['srgb'], rgbim=image_uint8, compat=crf_params['compat_bilateral'])
    Q = d.inference(5)
    refined_mask = np.argmax(Q, axis=0).reshape((image.shape[0], image.shape[1]))
    return refined_mask / 255.0

best_crf_params = {
    'sxy': 5,
    'compat': 3,
    'sxy_bilateral': 81,
    'srgb': 10,
    'compat_bilateral': 20
}

# Cálculo para operações morfológicas
def postprocess_mask_with_morphology(mask, morph_params):
    refined_mask = (mask * 255).astype(np.uint8)
    kernel = cv2.getStructuringElement(morph_params['shape'], morph_params['kernel_size'])
    if morph_params['operation'] == 'open':
        refined_mask = cv2.morphologyEx(refined_mask, cv2.MORPH_OPEN, kernel, iterations=morph_params['iterations'])
    elif morph_params['operation'] == 'close':
        refined_mask = cv2.morphologyEx(refined_mask, cv2.MORPH_CLOSE, kernel, iterations=morph_params['iterations'])
    elif morph_params['operation'] == 'dilate':
        refined_mask = cv2.dilate(refined_mask, kernel, iterations=morph_params['iterations'])
    elif morph_params['operation'] == 'erode':
        refined_mask = cv2.erode(refined_mask, kernel, iterations=morph_params['iterations'])
    return refined_mask / 255.0

best_morph_params = {
    'kernel_size': (3,3),
    'iterations': 1,
    'operation': 'dilate',
    'shape': cv2.MORPH_CROSS
}

def morphology_then_crf(image, mask, morph_params, crf_params):
    morph_mask = postprocess_mask_with_morphology(mask, morph_params)
    refined_mask = postprocess_mask_with_crf(image, morph_mask, crf_params)
    return refined_mask

In [ ]:
# Pós-processar as máscaras preditas com CRF denso
saidas_modelo_postprocessed = np.array([morphology_then_crf(images_test[i], saidas_novas_imagens_modelo[i].squeeze(), best_morph_params, best_crf_params) for i in range(len(saidas_novas_imagens_modelo))])

In [ ]:
# Converter as máscaras de teste para binário (0 e 1)
masks_test_binary = (np.array(masks_test) > 0).astype(np.uint8)
saidas_modelo_postprocessed_binary = (saidas_modelo_postprocessed > 0).astype(np.uint8)

# Calcular Jaccard Score
jaccard_scores = [jaccard_score(masks_test_binary[i].flatten(), saidas_modelo_postprocessed_binary[i].flatten(), average='binary') for i in range(len(masks_test))]
print(f"Jaccard Score: {np.mean(jaccard_scores)}")


In [ ]:
with tf.device('/gpu:0'):
    # Iterar sobre cada saída do modelo
    for i in range(len(images_test)):
        # Obter a entrada correspondente e a saída real
        img_entrada = masks_test[i]
        img_saida_real = saidas_novas_imagens_modelo[i]

        # Obter a saída gerada pelo modelo
        img_saida_modelo = saidas_modelo_postprocessed[i]

        # Mostrar as imagens
        plt.figure(figsize=(12, 4))

        plt.subplot(1, 3, 1)
        plt.imshow(img_entrada.squeeze(), cmap='gray')
        plt.title('saida real')
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.imshow(img_saida_real.squeeze(), cmap='gray')
        plt.title('Saída modelo')
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.imshow(img_saida_modelo.squeeze(), cmap='gray')
        plt.title('Saída do Modelo - pós')
        plt.axis('off')

        plt.show()